In [9]:
import pandas as pd
import numpy as np
from scipy.signal import argrelextrema
import matplotlib.pyplot as plt
import os

# Adjust the path as needed
data_path = '../../data/input/featured.csv'
df = pd.read_csv(data_path)

# Ensure date is datetime and sort
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

In [10]:
# -----------------------------
# SIMPLE PER-BAR TREND (Dow Theory)
# -----------------------------
prev_high = df['High'].shift(1)
prev_low = df['Low'].shift(1)
prev_close = df['Close'].shift(1)

higher_high = df['High'] > prev_high
lower_low = df['Low'] < prev_low
close_up = df['Close'] > prev_close
close_down = df['Close'] < prev_close

is_up = higher_high & close_up
is_down = lower_low & close_down

trend_spec = pd.Series('Sideways', index=df.index)
trend_spec[is_up] = 'Uptrend'
trend_spec[is_down] = 'Downtrend'

df['dow_trend_spec_raw'] = trend_spec
df['dow_trend_spec'] = trend_spec.replace('Sideways', np.nan).ffill().fillna('Sideways')

# -----------------------------
# MOTHER CANDLE LOGIC
# -----------------------------
df['range'] = df['High'] - df['Low']
df['avg_range'] = df['range'].rolling(window=20, min_periods=1).mean()
df['is_mother_candle'] = df['range'] > 1.1 * df['avg_range']

mother_candle_high = None
mother_candle_low = None
trend = 'Sideways'
mother_trend_list = []

for i in range(len(df)):
    if df.loc[i, 'is_mother_candle']:
        mother_candle_high = df.loc[i, 'High']
        mother_candle_low = df.loc[i, 'Low']
        trend = 'Sideways'
    elif mother_candle_high and df.loc[i, 'Close'] > mother_candle_high:
        trend = 'Uptrend'
    elif mother_candle_low and df.loc[i, 'Close'] < mother_candle_low:
        trend = 'Downtrend'

    mother_trend_list.append(trend)

df['mother_candle_trend'] = mother_trend_list

# -----------------------------
# COMBINE BOTH LOGICS
# -----------------------------
df['final_trend'] = np.where(
    df['dow_trend_spec_raw'].isin(['Uptrend', 'Downtrend']),
    df['dow_trend_spec_raw'],
    df['mother_candle_trend']
)

import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Example
df['final_trend'] = df['final_trend'].astype(str)

# Encode
encoder = LabelEncoder()
df['trend_code'] = encoder.fit_transform(df['final_trend'])
# downtrend = 0, uptrend = 1, sideways = 2

print(encoder.classes_)
print(df[['dow_trend_spec', 'trend_code']].head())

# -----------------------------
# VIEW FULL DATAFRAME
# -----------------------------
display_cols = ['date', 'Open', 'High', 'Low', 'Close', 'volume',  'dow_trend_spec_raw', 
                'mother_candle_trend', 'final_trend', 'trend_code','range', 'avg_range', 'is_mother_candle']

print(df[display_cols].to_string(index=False))

['Downtrend' 'Sideways' 'Uptrend']
  dow_trend_spec  trend_code
0       Sideways           1
1      Downtrend           0
2      Downtrend           1
3      Downtrend           0
4      Downtrend           0
                     date     Open     High      Low    Close     volume dow_trend_spec_raw mother_candle_trend final_trend  trend_code   range  avg_range  is_mother_candle
2013-08-01 00:00:00+05:30  5776.90  5808.50  5676.85  5727.85  216762208           Sideways            Sideways    Sideways           1  131.65 131.650000             False
2013-08-02 00:00:00+05:30  5750.05  5761.85  5649.00  5677.90  222494469          Downtrend            Sideways   Downtrend           0  112.85 122.250000             False
2013-08-05 00:00:00+05:30  5682.40  5721.00  5661.50  5685.40  197167952           Sideways            Sideways    Sideways           1   59.50 101.333333             False
2013-08-06 00:00:00+05:30  5664.90  5664.90  5521.80  5542.25  216227972          Downtrend        

In [11]:
# -----------------------------
# SELECT COLUMNS TO SAVE
# -----------------------------
candidate_cols_to_save = [
    'date', 'open', 'high', 'low', 'close', 'volume', 'final_trend', 'trend_code'
]

cols_present = [c for c in candidate_cols_to_save if c in df.columns]
df_to_save = df[cols_present].copy()

# -----------------------------
# SAVE CSV
# -----------------------------
output_path = '../../data/input/nifty-dow_theory_trend-2014to2025.csv'
df_to_save.to_csv(output_path, index=False)
print(f"Saved CSV with Mother Candle + Dow Theory trend: {output_path}")

# -----------------------------
# INSPECT TAIL FOR VERIFICATION
# -----------------------------
print("\n=== TAIL OF SAVED DATAFRAME ===\n")
print(df_to_save.tail(15).to_string(index=False))

# -----------------------------
# SAVE CSV
# -----------------------------
output_path = '../../data/input/full-featured.csv'
df.to_csv(output_path, index=False)
print(f"Saved Full Featured CSV with Mother Candle + Dow Theory trend: {output_path}")

# -----------------------------
# INSPECT TAIL FOR VERIFICATION
# -----------------------------
print("\n=== TAIL OF SAVED DATAFRAME ===\n")
print(df.tail(15).to_string(index=False))


Saved CSV with Mother Candle + Dow Theory trend: ../../data/input/nifty-dow_theory_trend-2014to2025.csv

=== TAIL OF SAVED DATAFRAME ===

                     date    volume final_trend  trend_code
2025-11-27 00:00:00+05:30 240287057     Uptrend           2
2025-11-28 00:00:00+05:30 202511545     Uptrend           2
2025-12-01 00:00:00+05:30 213828573   Downtrend           0
2025-12-02 00:00:00+05:30 254803113   Downtrend           0
2025-12-03 00:00:00+05:30 265660108   Downtrend           0
2025-12-04 00:00:00+05:30 248099545     Uptrend           2
2025-12-05 00:00:00+05:30 249254405     Uptrend           2
2025-12-08 00:00:00+05:30 320729920   Downtrend           0
2025-12-09 00:00:00+05:30 275978160   Downtrend           0
2025-12-10 00:00:00+05:30 207944446    Sideways           1
2025-12-11 00:00:00+05:30 206099631    Sideways           1
2025-12-12 00:00:00+05:30 216344338     Uptrend           2
2025-12-15 00:00:00+05:30 181557703   Downtrend           0
2025-12-16 00:00:00+05

In [12]:
# DOTTED BASELINE + COLORED SEGMENTS (bridges gaps)
print("\n=== DOTTED BASELINE + COLORED SEGMENTS ===\n")

trend_col = 'dow_trend_spec'
if trend_col not in df.columns:
    raise ValueError(f"Missing trend column: {trend_col}")

run_id = (df[trend_col] != df[trend_col].shift(1)).cumsum()
trend_colors = {'Uptrend': 'green', 'Downtrend': 'red', 'Sideways': 'gray'}

plt.figure(figsize=(16,6))
# 1) Draw full series as dotted gray baseline
plt.plot(df['date'], df['close'], color='lightgray', linewidth=1.2, linestyle='--', label='Baseline')

# 2) Overlay colored contiguous segments
for rid, seg in df.groupby(run_id):
    t = seg[trend_col].iloc[0]
    color = trend_colors.get(t, 'black')
    plt.plot(seg['date'], seg['close'], color=color, linewidth=2)

plt.title('NIFTY Close (Dotted Baseline + Colored Trend Segments)')
plt.xlabel('Date')
plt.ylabel('Close Price')
import matplotlib.patches as mpatches
handles = [mpatches.Patch(color=c, label=k) for k,c in trend_colors.items()]
handles.insert(0, mpatches.Patch(color='lightgray', label='Baseline'))
plt.legend(handles=handles)
plt.show()



=== DOTTED BASELINE + COLORED SEGMENTS ===



KeyError: 'close'

<Figure size 1600x600 with 0 Axes>